<a href="https://colab.research.google.com/github/ShriMLEngineer/furniture_classification/blob/main/model_app.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
!pip install flask

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
!git clone https://github.com/ShriMLEngineer/furniture_classification

Cloning into 'furniture_classification'...
remote: Enumerating objects: 428, done.
remote: Counting objects: 100% (117/117), done.
remote: Compressing objects: 100% (107/107), done.
remote: Total 428 (delta 27), reused 79 (delta 7), pack-reused 311
Receiving objects: 100% (428/428), 121.61 MiB | 26.32 MiB/s, done.
Resolving deltas: 100% (30/30), done.
Updating files: 100% (349/349), done.


In [18]:
from flask import Flask, request, jsonify
import numpy as np
from PIL import Image
import torch

app = Flask(__name__)

# Load the model
path = '/content/furniture_classification/detr-main'

model = torch.hub.load(path, 'detr_resnet50', source="local", pretrained=True, num_classes=4)
checkpoint = torch.load("/content/furniture_classification/outputs/checkpoint.pth", map_location='cpu')

model.load_state_dict(checkpoint['model'], strict=False)
model.eval();


# Define a route for image classification
@app.route('/predict', methods=['POST'])
def predict():
    # Get the image file from the request
    file = request.files['image']

    # Open the image file and preprocess it
    img = Image.open(file)
    img = img.resize((224, 224))
    img = np.array(img) / 255.0
    img = img.reshape(1, 224, 224, 3)

    # Use the model to make predictions
    preds = model.predict(img)

    # Convert the predictions to class labels
    classes = np.argmax(preds, axis=1)

    # Return the class label as a JSON response
    return jsonify({'class': int(classes[0])})

if __name__ == '__main__':
    app.run()

/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


INFO:werkzeug: * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
